In [ ]:
import pandas as pd
import re

# Load the Excel file
df = pd.read_excel("dataset_2_2.xlsx", header=None, names=["raw1"])

In [ ]:
# Drop missing values
df.dropna(inplace=True)

In [ ]:
# Split text and label at the last semicolon
df[['text1', 'label1']] = df['raw1'].str.rsplit(";", n=1, expand=True)

# Fill missing labels with empty strings
df.loc[:, 'label1'] = df['label1'].fillna('')

In [ ]:
# Apply cleaning
df["clean_text1"] = df["text1"].apply(clean_tweet)
df = df.sample(n=25, random_state=42)

In [ ]:
# Preview
print(df[["clean_text1", "label1"]].head())

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
from datasets import Dataset, load_metric

In [ ]:
# Prepare dataset1
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Encode texts
def tokenize(batch):
    return tokenizer(batch['clean_text1'], padding=True, truncation=True)

In [ ]:
# Convert DataFrame to Dataset
dataset1 = Dataset.from_pandas(df[['clean_text1', 'label1']])

In [ ]:
# Tokenize
dataset1 = dataset1.map(tokenize, batched=True)

In [ ]:
# Split train/test
train_test = dataset1.train_test_split(test_size=0.2)

In [ ]:
# Load model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(labels))

In [ ]:
# Training args
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
)

In [ ]:
# Define metric
metric = load_metric('f1')

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return metric.compute(predictions=preds, references=p.label_ids, average='macro')

In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test['train'],
    eval_dataset=train_test['test'],
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Predictions
preds_output = trainer.predict(train_test['test1'])
pred_labels = np.argmax(preds_output.predictions, axis=1)
true_labels = preds_output.label_ids

# Print metrics
print(classification_report(true_labels, pred_labels, target_names=labels))

# Confusion matrix
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(true_labels, pred_labels)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
model.save_pretrained("./emotion_bert_model")
tokenizer.save_pretrained("./emotion_bert_model")

In [ ]:
import streamlit as st
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import torch.nn.functional as F

@st.cache(allow_output_mutation=True)
def load_model():
    tokenizer = BertTokenizer.from_pretrained("./emotion_bert_model")
    model = BertForSequenceClassification.from_pretrained("./emotion_bert_model")
    model.eval()
    return tokenizer, model

tokenizer, model = load_model()

st.title("Tweet Emotion Detection")

user_input = st.text_area("Enter Tweet Text:")

if st.button("Predict Emotion"):
    inputs = tokenizer(user_input, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
    pred_idx = torch.argmax(probs).item()
    pred_prob = probs[0][pred_idx].item()
    emotion = list(label2id.keys())[list(label2id.values()).index(pred_idx)]

    st.write(f"**Predicted Emotion:** {emotion}")
    st.write(f"**Confidence:** {pred_prob:.2f}")

    st.write("**Probabilities:**")
    for i, label in enumerate(label2id.keys()):
        st.write(f"{label}: {probs[0][i].item():.2f}")
